##  Nerual Network With Single Hidden Layer 

**README**: This derivation and go-through notes is made based on my own mathematical proof and may contain errors here or there. If you find one, don't hesitate to reach me out and point out where I have made mistakes.

Email: yd1@andrew.cmu.edu

This notebook will go through the math derivation of forward and backpropogation of the nerual network with one hidden layer

<img src="images/nn_1.png" style="width:600px;height:300px;">

**Background**:
In this figure, we have a single training example that is connected with 3 neuron (hidden layer) and combined together to output the output layer

**Clarification**:
The following are some of the notations or functions that will be used in the derivation (continously adding)
    - Activation function for hidden layer is tanh, and output layer is sigmoid (for the reason, please refer to Prof.Andrew Ng's lecture notes)
    - Capital letters refer to the matrices that spans over the entire training set.
    - m : number of training examples (size)
    - n_x : feature dimension of a single training example
    - n_h : size of hidden layer
    - n_y : size of output layer (for binary classfication n_y = 2)
    - i : i th training example
    - square bracket : the nth layer (i.e. [1]} means the hidden layer)
    - loss function - cross entropy (for classification)
    - cost function (mean loss):
<br>
$$J = - \frac{1}{m}\sum_{i=1}^{m}(y^{(i)}log(a^{[2](i)}) + (1 - y^{(i)})log(1 - a^{[2](i)}))$$
<br>

So the whole model is really well represented by the following picture (by Prof.Andrew Ng), even though the dimension may not consistent with this note.
<img src="images/nn_2.png" style="width:600px;height:300px;">

### Forward Propogation
<br>
$$Z^{[1]} = W^{[1]} X + b^{[1]}$$
<br>
$$A^{[1]} = g{(Z^{[1]})}$$
<br>
$$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$$
<br>
$$A^{[2]} = g{(Z^{[2]})}$$

A lot of bugs in the code are caused by incorrect dimension of the matrix, so it is important to check the dimension throughout the calculation.


**Sanity Check: Dimensionality**

- First layer (hidden layer)
$$X : (n_x, m)$$
$$W^{[1]} : (n_h, n_x)$$
$$W^{[1]} \cdot X : (n_h, m)$$
$$b^{[1]} : (n_h, m)$$
$$Z^{[1]}, A^{[1]} : (n_h, m)$$

- output layer
$$W^{[2]} : (n_y, n_h)$$
$$W^{[1]} \cdot X : (n_y, m)$$
$$b^{[2]} : (n_y, m)$$
$$Z^{[2]}, A^{[2]} : (n_y, m)$$

now, we have m training examples and each of them output consists of the $\textbf{probability}$ of two binary classfication results, and combined with

$$y^{(i)}_{prediction} = \begin{cases} 1 & if a^{[2](i)} > 0.5 \\ 0 & otherwise\end{cases}\tag{5}$$

can produce a (1, m) matrix with predicted $\hat y$

In [1]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed() 
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
# sigmoid activation function
def sigmoid(s):
    return 1 / (1 + np.exp(-s))

In [ ]:
def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """

    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    
    # Implement Forward Propagation to calculate A2 (probabilities)
    Z1 = np.dot(W1, X) + b1
    # Use tanh for the activation function in the hidden layer
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

### Compute the cost


(you can use either `np.multiply()` and then `np.sum()` or directly `np.dot()`).  
Note that if you use `np.multiply` followed by `np.sum` the end result will be a type `float`, whereas if you use `np.dot`, the result will be a 2D numpy array.  We can use `np.squeeze()` to remove redundant dimensions (in the case of single float, this will be reduced to a zero-dimension array). We can cast the array as a type `float` using `float()`.

In [3]:
# compute the cost of training 
def compute_cost(A2, Y):
    """
    Computes the cross-entropy cost given in equation (1)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    cost -- cross-entropy cost given equation (1)
    """
    
    m = Y.shape[1] # number of example

    # Compute the cross-entropy cost
    logprobs = np.multiply(np.log(A2), Y) + np.multiply(np.log(1 - A2), (1 - Y))
    cost = - 1/m * (np.sum(logprobs, axis=1, keepdims=True)) 
    
    cost = float(np.squeeze(cost))  # makes sure cost is the dimension we expect. 
                                    # E.g., turns [[17]] into 17 
        
    assert(isinstance(cost, float))
    
    return cost

### Back Propogation -- Gradient Descent **

**Clarification** 
The following derivation will denote derivative of $\mathcal{L}$ respect to any variable as $\partial {var}$ i.e. 
$\frac{\partial \mathcal{L}}{\partial a} := \partial a$
$$\mathcal{L}(A, Y) = -Y\log(A) - (1 - A) \log(1 - A)$$ 
<br>
$$\partial A^{[2]} = \frac{\partial \mathcal{L}}{\partial A} = -\frac{y}{A} +\frac{1-Y}{1-A}$$

then, use chain rule to calculat the $\partial Z^{[1]}$, capital letters represent matrix over all training examples
<b>Note</b> here, for output layer our activation function is still sigmoid.
$$\partial Z^{[2]} = \frac{\partial \mathcal{L}}{\partial A^{[2]}}*\frac{\partial A^{[2]}}{\partial Z^{[2]}}$$
<br>
$$ =  (-\frac{Y}{A^{[2]}} +\frac{1 - Y}{1 - A^{[2]}})*(\sigma(Z^{[2]})(1 - \sigma(Z^{[2]})) )$$
<br>
$$ = (-\frac{Y}{\sigma(Z^{[2]})} +\frac{1 - Y}{1-\sigma(Z^{[2]})})*(\sigma(Z^{[2]})(1 - \sigma(Z^{[2]})))$$
<br>
$$ = -Y(1 - \sigma(Z^{[2]})) + \sigma(Z^{[2]})(1 - Y)$$
<br>
$$ = \sigma(Z^{[2]}) - Y = A^{[2]} - Y $$

now, its time to compute $\partial W^{[2]}$ and $\partial b^{[2]}$
$$ \partial W^{[2]} = \frac{\partial \mathcal{L}}{\partial Z^{[2]}} \frac{\partial Z^{[2]}}{\partial W^{[2]}} = \partial Z^{[2]} \frac{\partial Z^{[2]}}{\partial W^{[2]}} = \partial Z^{[2]} A^{[1] T}$$

<b>Note</b>, the transpose of $A^{[1]}$ is because the $\partial Z^{[2]}$ is a $(n_y, m)$ matrix and $A^{[1]}$ is $(n_h, m)$ matrix. In order to perform matrix multiplication, need to transpose the $A^{[2]}$, and now $\partial W^{[2]}$ is a $(n_y, n_h)$ matrix. Coherent withprevious $W^{[2]}$'s dimension.

Similarly:
$$ \partial b^{[2]} = \frac{\partial \mathcal{L}}{\partial Z^{[2]}} \frac{\partial Z^{[2]}}{\partial b^{[2]}} = \partial Z^{[2]} \frac{\partial Z^{[2]}}{\partial b^{[2]}} = \partial Z^{[2]}$$




Now, let's compute the $\partial Z^{[1]}$ (p.s. I think this is the most difficult part in the derivation)

$$ \partial Z^{[1]} = \frac{\partial \mathcal{L}}{\partial Z^{[2]}}\frac{\partial Z^{[2]}}{\partial A^{[1]}}\frac{\partial A^{[1]}}{\partial Z^{[1]}}$$

 Since in the middle hidden layer the activation we implemented will be tanh, thus let $ g(x) = tanh(x)$, therefore, $A^{[1]} = g(Z^{[1]}) = tanh(Z^{[1]})$, and corresponding derivitive is $g^{'} = 1 - Z^{[1]2}$ and '*' denotes the element-wise multiplication, in numpy its `np.multiply`

$$ = W^{[2]T} \partial Z^{[2]} * (g'(Z^{[1]}))$$
<br>
Up to this point, it is again very necessary to do a sanity check of dimension to make sure the matrices have right dimensions.<br>
$$ Z^{[1]}, dZ^{[1]} : (n_h, m)$$
<br>
$$ Z^{[2]}, dZ^{[2]} : (n_y, m)$$
<br>
$$dW^{[2]} : (n_y, n_h)$$
<br>
$$ dim (W^{[2]T}\partial Z^{[2]} * (g'(Z^{[1]})) = (n_y, n_h)^{T}(n_y, m) * (n_h, m) $$
<br>
$$ = (n_h, m) = dim(Z^{[1]})$$

Lastly, compute $\partial W^{[1]}$ and $\partial b^{[1]}$
$$ \partial W^{[1]} = \frac{\partial \mathcal{L}}{\partial Z^{[1]}} \frac{\partial Z^{[1]}}{\partial W^{[1]}} = \partial Z^{[1]} X^{[T]}$$
<br>
$$ \partial b^{[1]} = \frac{\partial \mathcal{L}}{\partial Z^{[1]}} \frac{\partial Z^{[1]}}{\partial b^{[1]}} = \partial Z^{[1]} \frac{\partial Z^{[1]}}{\partial b^{[1]}} = \partial Z^{[1]}$$

In [ ]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation
    
    Arguments:
    parameters -- python dictionary containing parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    W1 = parameters["W1"]
    W2 = parameters["W2"]
        
    # Retrieve also A1 and A2 from dictionary "cache".
    A1 = cache["A1"]
    A2 = cache["A2"]
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    dZ2 = A2 - Y
    dW2 = 1/m * np.dot(dZ2, A1.T)
    db2 = 1/m * np.sum(dZ2, axis = 1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), (1 - np.power(A1,2))) 
    dW1 = 1/m * np.dot(dZ1, X.T)
    db1 = 1/m * np.sum(dZ1, axis = 1, keepdims=True)
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

### Update the parameters

**General gradient descent rule**: $ \theta = \theta - \alpha \frac{\partial J }{ \partial \theta }$ where $\alpha$ is the learning rate and $\theta$ represents a parameter.

In [3]:
def update_parameters(parameters, grads, learning_rate = 1.2):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Retrieve each gradient from the dictionary "grads"
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    # Update rule for each parameter
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

### Integration of the Model 

In [4]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Initialize parameters
    parameters = initialize_parameters(n_x, n_h, n_y)    
    # Loop (gradient descent)

    for i in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X, parameters)
        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A2, Y, parameters)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads)
                
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

### Prediction 

**Reminder**:
predictions = $y_{prediction}$ = $
\begin{cases}
      1 & \text{if}\ activation > 0.5 \\
      0 & \text{otherwise}
\end{cases}$  
    
As an example, if you would like to set the entries of a matrix X to 0 and 1 based on a threshold you would do: ```X_new = (X > threshold)```

In [2]:
def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    A2, cache = forward_propagation(X, parameters)
    predictions = (A2 > 0.5)    
    return predictions

Reference:
- http://scs.ryerson.ca/~aharley/neural-networks/
- http://cs231n.github.io/neural-networks-case-study/